In [21]:
import tkinter as tk
from tkinter import messagebox, simpledialog
import threading
import time
import random

class DistributedSystemGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Distributed File System with Fault Tolerance + CPU Scheduling")
        
        self.nodes = {"Node1": True, "Node2": True, "Node3": True}  # True = Online, False = Offline
        self.node_labels = {}
        self.process_queue = []

        self.setup_gui()

    def setup_gui(self):
        # Node status labels
        for idx, node in enumerate(self.nodes):
            label = tk.Label(self.root, text=f"{node}: Online", fg="green", font=('Arial', 12, 'bold'))
            label.grid(row=0, column=idx, padx=10, pady=10)
            self.node_labels[node] = label

        # Control buttons
        tk.Button(self.root, text="Distribute File", command=self.distribute_file).grid(row=1, column=0, pady=10)
        tk.Button(self.root, text="Simulate Node Failure", command=self.simulate_node_failure).grid(row=1, column=1, pady=10)
        tk.Button(self.root, text="Revive Node", command=self.revive_node).grid(row=1, column=2, pady=10)
        tk.Button(self.root, text="Add Process", command=self.add_process).grid(row=2, column=0, pady=10)
        tk.Button(self.root, text="Run Round Robin", command=self.run_round_robin).grid(row=2, column=1, pady=10)
        tk.Button(self.root, text="Export Log", command=self.export_log).grid(row=2, column=2, pady=10)

        # Log output
        self.log_box = tk.Text(self.root, height=15, width=60)
        self.log_box.grid(row=3, column=0, columnspan=3, padx=10, pady=10)

    def log(self, message):
        timestamp = time.strftime("[%H:%M:%S] ")
        self.log_box.insert(tk.END, timestamp + message + "\n")
        self.log_box.see(tk.END)

    def distribute_file(self):
        self.log("Starting file distribution...")
        for node in self.nodes:
            if self.nodes[node]:
                self.log(f"✅ File distributed to {node}")
            else:
                self.log(f"❌ File NOT distributed to {node} (Offline)")

    def simulate_node_failure(self):
        online_nodes = [node for node, status in self.nodes.items() if status]
        if not online_nodes:
            messagebox.showinfo("Info", "All nodes are already offline!")
            return
        failed_node = random.choice(online_nodes)
        self.nodes[failed_node] = False
        self.node_labels[failed_node].config(text=f"{failed_node}: Offline", fg="red")
        self.log(f"⚠️ Simulated failure: {failed_node} is now Offline")

    def revive_node(self):
        offline_nodes = [node for node, status in self.nodes.items() if not status]
        if not offline_nodes:
            messagebox.showinfo("Info", "No offline nodes to revive.")
            return

        node_to_revive = simpledialog.askstring("Revive Node", f"Offline nodes: {', '.join(offline_nodes)}\nEnter node name to revive:")
        if node_to_revive in self.nodes and not self.nodes[node_to_revive]:
            self.nodes[node_to_revive] = True
            self.node_labels[node_to_revive].config(text=f"{node_to_revive}: Online", fg="green")
            self.log(f"♻️ {node_to_revive} has been revived and is now Online.")
        else:
            messagebox.showerror("Error", "Invalid or already online node selected.")

    def add_process(self):
        process_name = simpledialog.askstring("Add Process", "Enter process name:")
        if process_name:
            self.process_queue.append(process_name)
            self.log(f"🆕 Process '{process_name}' added to queue.")

    def run_round_robin(self):
        if not self.process_queue:
            messagebox.showinfo("Info", "No processes in queue.")
            return

        def rr_scheduler():
            self.log("⚙️ Starting Round Robin Scheduling (Time Slice = 2s)")
            while self.process_queue:
                process = self.process_queue.pop(0)
                self.log(f"⏳ Running process: {process}")
                time.sleep(2)  # Simulate time slice
                if random.choice([True, False]):  # Randomly mark process as done or re-queue
                    self.log(f"✅ Process {process} completed.")
                else:
                    self.log(f"🔁 Process {process} not finished. Re-queued.")
                    self.process_queue.append(process)
            self.log("🎉 All processes completed.")

        threading.Thread(target=rr_scheduler).start()

    def export_log(self):
        try:
            log_text = self.log_box.get("1.0", tk.END)
            with open("log_output.txt", "w") as file:
                file.write(log_text)
            messagebox.showinfo("Success", "Log exported to 'log_output.txt'")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to export log: {e}")

if __name__ == "__main__":
    root = tk.Tk()
    app = DistributedSystemGUI(root)
    root.mainloop()
